In [1]:
import pandas as pd
import yfinance as yf
import sqlalchemy as db
import pyodbc

In [2]:
tickers = ['JOE', 'NEM', 'ASC', 'INVA', 'TRIP', 'JBLU', 'GEN', 'PLTR', 'ACHR', 'BAC']
start_date = '2024-01-01'

data = yf.download(tickers, start=start_date, interval='1d', group_by='ticker', auto_adjust=True, actions=True)

# Concatenate all the dataframes into one
df = pd.concat([data[ticker] for ticker in tickers], keys=tickers)

# Reset index to get 'Date' as a column instead of index
df.reset_index(inplace=True)

# Rename the 'level_0' column to 'Ticker'
df.rename(columns={'level_0': 'Ticker'}, inplace=True)

# Reorder columns as per your requirement
df = df[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']]

print(df)


[*********************100%***********************]  10 of 10 completed
    Ticker       Date       Open       High        Low      Close    Volume  \
0      JOE 2024-01-02  59.926211  60.445053  58.090310  58.698952    170600   
1      JOE 2024-01-03  58.190085  58.349729  56.663495  57.711155    177200   
2      JOE 2024-01-04  57.631332  58.688972  57.282113  57.391869    143700   
3      JOE 2024-01-05  57.002740  58.030444  56.892984  57.302071    120900   
4      JOE 2024-01-08  57.461712  58.659039  57.112493  58.519352    166100   
..     ...        ...        ...        ...        ...        ...       ...   
705    BAC 2024-04-08  37.200001  37.580002  37.130001  37.500000  29470300   
706    BAC 2024-04-09  37.509998  37.770000  37.200001  37.730000  30977700   
707    BAC 2024-04-10  37.180000  37.230000  36.549999  36.650002  37610700   
708    BAC 2024-04-11  36.750000  36.750000  36.049999  36.349998  38604000   
709    BAC 2024-04-12  35.810001  36.090000  35.529999  35.6

In [3]:
df.rename(columns={'Stock Splits': 'StockSplits'}, inplace=True)
df

,Ticker,Date,Open,High,Low,Close,Volume,Dividends,StockSplits
0,JOE,2024-01-02,59.926211,60.445053,58.090310,58.698952,170600,0.0,0
1,JOE,2024-01-03,58.190085,58.349729,56.663495,57.711155,177200,0.0,0
2,JOE,2024-01-04,57.631332,58.688972,57.282113,57.391869,143700,0.0,0
3,JOE,2024-01-05,57.002740,58.030444,56.892984,57.302071,120900,0.0,0
4,JOE,2024-01-08,57.461712,58.659039,57.112493,58.519352,166100,0.0,0
...,...,...,...,...,...,...,...,...,...
705,BAC,2024-04-08,37.200001,37.580002,37.130001,37.500000,29470300,0.0,0
706,BAC,2024-04-09,37.509998,37.770000,37.200001,37.730000,30977700,0.0,0
707,BAC,2024-04-10,37.180000,37.230000,36.549999,36.650002,37610700,0.0,0
708,BAC,2024-04-11,36.750000,36.750000,36.049999,36.349998,38604000,0.0,0


In [4]:
from sqlalchemy import create_engine
import urllib

# Your connection parameters are formatted into a URL
params = urllib.parse.quote_plus(
    "Driver={SQL Server};"
    "Server=;"  # Make sure 'server' is replaced by your actual server name
    "Database=;"
    "Trusted_Connection=yes;"
)

# Create an SQLAlchemy engine using the connection URL
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [5]:
df.to_sql('Stocks', con=engine, if_exists='append', index=False)

In [7]:
df.to_csv("df.csv")